# Summary

# Business Understanding

# Data Understanding

### Importing relevant libraries

In [16]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading the dataset


In [14]:
encodings = ['utf-8', 'ISO-8859-1', 'Windows-1252', 'utf-16']
for encoding in encodings:
    try:
        df = pd.read_csv(r'C:\Users\user\Documents\MORINGA\Phase4\Project\judge-1377884607_tweet_product_company.csv', encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except Exception as e:
        print(f"Error with encoding {encoding}: {e}")

Error with encoding utf-8: 'utf-8' codec can't decode byte 0x89 in position 54: invalid start byte
Successfully read the file with encoding: ISO-8859-1


In [15]:
# Printing the first five rows of the data set
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Data Cleaning

In [17]:
# Checking for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


* There are 9093 rows and three columns in our dataset.
* There are missing values in all in a number of rows.

In [24]:
# checking for any pattern in the missing values
# emotion_in_tweet_is directed_at column
df['emotion_in_tweet_is_directed_at'].value_counts()


emotion_in_tweet_is_directed_at
iPad                               6748
Apple                               661
iPad or iPhone App                  470
Google                              430
iPhone                              297
Other Google product or service     293
Android App                          81
Android                              78
Other Apple product or service       35
Name: count, dtype: int64